In [1]:
import duckdb
import pandas as pd
import warnings

warnings.simplefilter(action='ignore')
pd.set_option('display.max_colwidth', None)
pd.set_option("max_colwidth", None)
pd.set_option("max_seq_items", None)
pd.set_option("display.max_columns", 78)
pd.set_option('display.max_rows', 500)

%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%config SqlMagic.named_parameters="disabled"
%sql duckdb:///:memory:


Deploy Dash apps for free on Ploomber Cloud! Learn more: https://ploomber.io/s/signup


In [5]:
%sql SELECT COUNT(*) FROM "../data/raw/race_results.parquet"

,count_star()
0,58238


In [8]:
%%sql --save race_results
race_results << 
select 
    case nullif(SexoT10, 'nan') when 'M' then 'Male' when 'F' then 'Female' end as gender,
    try_cast(nullif(MiliSegCarreraT05, 'nan') as int) as time_ms,
    to_milliseconds(time_ms) as time_interval,
    printf(
        '%02d:%02d:%02d', 
        extract(hours from time_interval), 
        extract(minutes from time_interval), 
        extract(seconds from time_interval)
    ) AS formatted_time,
    year(FechaCarreraT01) as race_year,
    regexp_replace(NombreCarreraT01, '\s+', ' ', 'g') as race_name,
    FechaCarreraT01 as race_date,
    try_cast(DistanciaMetrosTotalT01 as int) as race_distance,
    time_ms / race_distance as pace_ms_per_m,
    to_milliseconds(pace_ms_per_m * 1000 )as pace_per_km,
    printf('%02d:%02d', extract(minutes from pace_per_km),  extract(seconds from pace_per_km)) AS pace_min_per_km_formatted,
    strptime(left(nullif(FechaNacimientoT05, 'nan'), 10), '%d/%m/%Y') as dob,
    datediff('year', dob, FechaCarreraT01) - 1 as age,
    time_ms is null as is_finisher,
    case 
        when age < 18 then '<18'
        when age between 18 and 24 then '18-24'
        when age between 25 and 39 then '26-39'
        when age between 40 and 49 then '40-49'
        when age between 50 and 59 then '50-59'
        when age between 60 and 69 then '60-69'
        when age > 69 then '>69'
    end as category,
    NombreT11 as team_name

from "../data/raw/race_results.parquet"

## Fastest races (5k-8k)

In [14]:
%%sql
select
    race_name,
    race_year,
    gender,
    race_distance,
    to_milliseconds(median(pace_ms_per_m) * 1000 ) as median_pace_per_km,
    printf('%02d:%02d', extract(minutes from median_pace_per_km),  extract(seconds from median_pace_per_km)) AS median_pace_min_per_km_formatted,
    count(*) as num_participants

from race_results
where race_distance >= 5000 and race_distance < 8000
group by 1,2,3,4
having num_participants > 30
order by 5
limit 10


Generating CTE with stored snippets: 'race_results'

,race_name,race_year,gender,race_distance,median_pace_per_km,median_pace_min_per_km_formatted,num_participants
0,SAN SILVESTRE MEMBRILLA MANGA 1 - 2018,2018,Male,5000,0 days 00:03:56.746200,03:56,85
1,CROSS POPULAR - CAMPING CARAVANING 2022,2022,Male,6000,0 days 00:03:58.666667,03:58,51
2,SAN SILVESTRE MEMBRILLA MANGA 1,2018,Male,5000,0 days 00:04:01.557800,04:01,96
3,CAMPO A TRAVÉS - CAMPING 2023 - POPULAR,2023,Male,6000,0 days 00:04:02.588917,04:02,43
4,SAN SILVESTRE MEMBRILLA MANGA 1 - 2021,2021,Male,5000,0 days 00:04:04.960200,04:04,68
5,CAMPO A TRAVÉS - TARANCÓN 2023,2023,Male,5000,0 days 00:04:09.957400,04:09,36
6,SAN SILVESTRE MEMBRILLA - ABSOLUTA MAS - 2021,2021,Male,5000,0 days 00:04:10.265600,04:10,58
7,CROSS POPULAR - VILLAR DE OLALLA 2022,2022,Male,5400,0 days 00:04:12.685185,04:12,48
8,CROSS POPULAR - JUAN CARLOS DE LA OSSA Tarancon 2022,2022,Male,5000,0 days 00:04:13.117500,04:13,50
9,S.SILVESTRE MEMBRILLA - ABS 2018,2018,Male,5000,0 days 00:04:14.685400,04:14,85


## Fastest races (8k-15k)

In [13]:
%%sql
select
    race_name,
    race_year,
    gender,
    race_distance,
    median(pace_ms_per_m) as median_pace_ms_per_m,
    to_milliseconds(median_pace_ms_per_m * 1000 )as median_pace_per_km,
    printf('%02d:%02d', extract(minutes from median_pace_per_km),  extract(seconds from median_pace_per_km)) AS median_pace_min_per_km_formatted,
    count(*) as num_participants

from race_results
where race_distance >= 8000 and race_distance < 15000
group by 1,2,3,4
having num_participants > 30
order by 5
limit 10


Generating CTE with stored snippets: 'race_results'

,race_name,race_year,gender,race_distance,median_pace_ms_per_m,median_pace_per_km,median_pace_min_per_km_formatted,num_participants
0,XVI TROFEO SAN GIL - CCPP 2018 DIP. DE CUENCA,2018,Male,8000,253.151625,0 days 00:04:13.151625,04:13,228
1,V CARRERA POPULAR DE LA GUITARRA ESPAÑOLA,2021,Male,9500,253.571684,0 days 00:04:13.571684,04:13,97
2,XVIII TROFEO SAN GIL - MOTILLA,2022,Male,8000,258.282125,0 days 00:04:18.282125,04:18,200
3,XVII TROFEO SAN GIL,2019,Male,8000,266.949500,0 days 00:04:26.949500,04:26,258
4,VII CARRERA POPULAR VILLARRUBIA DE LOS OJOS - 10K,2019,Male,10000,267.234100,0 days 00:04:27.234100,04:27,184
5,XVII CARRERA POPULAR VILLA DE VILLAMAYOR DE SANTIAGO - CCPP 2018 DIP. DE CUENCA,2018,Male,10000,267.711000,0 days 00:04:27.711000,04:27,312
6,XXXV CARRERA POPULAR CROSS EL CAÑO - XIV MEMORIAL J.A. SEQUÍ - CCPP 2018 DIP. DE CUENCA,2018,Male,10000,267.826800,0 days 00:04:27.826800,04:27,359
7,6º C. P. VILLARUBIA DE LOS OJOS - 10K,2018,Male,10000,268.349600,0 days 00:04:28.349600,04:28,164
8,IV CARRERA POPULAR TIERRA DEL VÍTOR - CCPP 2018 DIP. DE CUENCA,2018,Male,10000,268.691000,0 days 00:04:28.691000,04:28,153
9,V CARRERA POPULAR - TIERRA DEL VÍTOR,2019,Male,10000,270.172200,0 days 00:04:30.172200,04:30,159


## Fastest races (>15k)

In [14]:
%%sql
select
    race_name,
    race_year,
    gender,
    race_distance,
    median(pace_ms_per_m) as median_pace_ms_per_m,
    to_milliseconds(median_pace_ms_per_m * 1000 )as median_pace_per_km,
    printf('%02d:%02d', extract(minutes from median_pace_per_km),  extract(seconds from median_pace_per_km)) AS median_pace_min_per_km_formatted,
    count(*) as num_participants

from race_results
where race_distance > 15000
group by 1,2,3,4
having num_participants > 30
order by 5 desc
limit 10


Generating CTE with stored snippets: 'race_results'

,race_name,race_year,gender,race_distance,median_pace_ms_per_m,median_pace_per_km,median_pace_min_per_km_formatted,num_participants
0,LA OSERA 21K,2023,Female,21000,559.728429,0 days 00:09:19.728429,09:19,31
1,LA OSERA 21K,2023,Male,21000,521.965619,0 days 00:08:41.965619,08:41,130
2,ZENAGAS TRAIL 30K - 2023,2023,Male,30000,497.621083,0 days 00:08:17.621083,08:17,111
3,MEDIA MARATÓN LA SARGA 2023,2023,Male,19000,484.681447,0 days 00:08:04.681447,08:04,71
4,CXM PALOMERA 2023 - 22K,2023,Male,22000,380.438841,0 days 00:06:20.438841,06:20,49
5,10 MILLAS EL PICAZO,2022,Female,16090,347.349782,0 days 00:05:47.349782,05:47,36
6,MEDIA MARATON DE CUENCA - 2022,2022,Female,21000,343.138381,0 days 00:05:43.138381,05:43,47
7,MEDIO MARATHON RUIDERA - Maravilla Natural 2022,2022,Female,21000,330.553048,0 days 00:05:30.553048,05:30,43
8,MEDIA MARATON CUENCA 2023 - GRAN PREMIO DE OTOÑO,2023,Female,21000,322.215643,0 days 00:05:22.215643,05:22,47
9,21K - MEDIA CUENCA 2019 - 21K,2019,Female,21000,318.652476,0 days 00:05:18.652476,05:18,61


## Circuit races

In [28]:
%%sql

with circuit_races as (

    select * from read_csv('../data/circuit_races.csv')

)

select 
    race_name,
    race_number,
    race_location,
    count(*) as num_participants,
    count_if(is_finisher is not null)::int as num_finishers

from race_results
left join circuit_races using (race_name, race_year)
group by 1,2,3
order by race_number
limit 5;


Your query is using one or more of the following snippets: race_results. JupySQL does not support snippet expansion within CTEs yet, CTE generation is disabled

,race_name,race_number,race_location,num_participants,num_finishers
0,VII C.P. DE LA GUITARRA ESPAÑOLA - 2023,1,Casasimarro,354,354
1,VI CARRERA POPULAR DE LA GUITARRA ESPAÑOLA - CASASIMARRO,1,Casasimarro,257,257
2,VILLARTA - CIRCUITO CCPP 2024,1,Villarta,528,528
3,XIII 10K LA PITA - EL HERRUMBLAR 2022,2,El Herrumblar,229,229
4,VIII Carrera Popular de la Guitarra Española - CIRCUITO CCPP 2024,2,Casasimarro,299,299
